# 0 摘要

BERT全称Bidirectional Encoder Representations from Transformers. BERT模型旨在基于无标签文本的上下文来预训练词的深度双向表征。预训练好的BERT模型可以简单地在下游接一层输出层，从而在不同任务上达到state-of-art的预测效果，例如QA、语言推断。

# 1 简介

在NLP任务中，预训练词向量被证明能够起到很好的作用。在实际应用中，有两种将预训练向量用在下游任务的方式：

- 1.**feature-based**。例如ELMo使用task-specific的模型结构将预训练向量作为额外的特征；
- 2.**fine-tuning**。例如GPT引入极少的task-specific参数并在下游任务训练过程中对词向量做fine-tuning。

上面这两种方法在预训练过程中都使用了相同的目标函数，即使用单向语言模型(unidirectional Language)去学习通用的语言表征。例如在GPT中，作者使用了从左到右的单向结构，使得每个token在做self-attention的时候都只能看到之前的token（参考Transformer的Decoder），这种单向模型叫做masked language model(MLM)；这样的限制使得token无法获取上下文完整信息，对于学习的词向量有极大的影响，尤其在QA这种强依赖上下文信息的任务中。

在这篇论文中，作者提出了BERT，通过两个子任务缓解了之前提到的MLM问题：

- 1.随机对一个句子中的某个token进行遮盖，利用上下文信息来预测这个token；
- 2."Next Sentence Prediction"任务，来联合预训练text-pair（相邻句子上下文信息）的表征。

论文的主要贡献有是三个方面：

- 1.证明了Bidirectional pre-training for language representions的重要性；
- 2.pre-trained representations可以大量减少下游复杂的模型架构和参数；
- 3.BERT刷新了11项NLP任务的记录。

# 2 相关工作

## 2.1 Unsupervised Feature-based Approaches

# 3 BERT概况

## 3.1 模型简介

BERT框架共分为两个阶段：

- 1.pre-training阶段。在这一阶段，BERT基于不同的预训练任务进行学习获得token向量；
- 2.fine-tuning阶段。利用pre-training阶段训练好的参数初始化，接入不同的有监督下游任务。

**BERT最显著的特点就是是一个跨任务的统一架构（Unified architecture across different tasks）。即BERT结构和其下游任务结构之间差异很小**

## 3.2 模型结构

BERT模型结构是一个多层双向Transformer Encoder。在论文中，定义Transformer Block层数为$L$，隐藏层结点数为$H$，self-attention heads为$A$。

BERT发布了两个结构：

- BERT Base: L = 12, H = 768, A=12, 总参数量为110M
- BERT Large: L = 24, H = 1024, A=16, 总参数量340M

> BERT Base主要是用来和同样模型规模的GPT进行比较，BERT Base采用双向的self-attention，即是Encoder结构；GPT采用的是单向self-attention，是Decoder结构。

## 3.3 输入与输出表示

为了让BERT能够跨任务学习，应用于不同的下游任务，我们的输入表示（Input Representations）能够在一个标记序列中明确地表示单个句子和一对句子（例如<Question, Answer>）。

在BERT中，一个"sentence"是指任意一段连续的文本，并非指自然句。一个"sequence"是指输入给BERT的token序列，既可以是一个句子，也可以两个句子一起。

每个sequence的第一个token永远是classification token（$[CLS]$），$[CLS]$的最后一个隐藏层状态用来作为下游分类任务的sentence embedding。

Sentence Pairs被组合为一个sequence，通过以下方式对两个句子进行区分：

- 用$[SEP]$ token区分两个句子；
- 对输入sequence的每个token加入一个learned embedding用来区分这个token到底是属于句子A还是句子B。

对于给定的token，它的represention由对应的token/segment/position embedding求和而得，如下图所示：

![](pictures/bert-input-representations.png)

# 4 BERT细节